In [1]:
from bs4 import BeautifulSoup as bs
import pickle
import pandas as pd
import numpy as np

In [2]:
with open('baseball_data.pkl','rb') as cellar:
    season_html = pickle.load(cellar)
len(season_html)

119

In [3]:
type(season_html)

list

These pages are so incredibly gross. I've scraped a lot of pig shit (literally) in my life, and I prefer that to this.

The tables of interest have these headers:
* MLB Detailed Standings
* Team Standard Batting
* Team Standard Pitching
* MLB Wins Above Avg By Position
* Team Fielding

I envision a dataframe storing the league summary for each statistic (columns) by year (rows). I envisioned a numpy array of three dimensions storing data by team (thickness...). However, I suspect a better approach would be to create a dictionary of dataframes, one for each team. baseball-reference uses a three letter code for each team, although it does change when a franchise moves. I don't anticipate this being a problem, and the number of moves is not so overpowering as to prevent concatenating the dataframes into single entries later if I thought it useful.

In [4]:
season_html[0].text.find('MLB  Detailed Standings')

70486

In [5]:
# set up a dictionary to call seasons by number
# first year is 1901
# format data via BeautifulSoup
season_soup = {}
for i, season in enumerate(season_html):
    cleaned_text = season.text.replace("<!--","").replace("-->","")
    season_soup[1901+i] = bs(cleaned_text)

In [6]:
season_soup[1982].find('a')

<a href="https://www.sports-reference.com/"><svg height="15px" width="20px"><use xlink:href="#ic-sr-pennant"></use></svg> Sports Reference</a>

The analysis I want to conduct is time-based, so I want to have dataframes for the big leagues as a whole, maybe the individual leagues, and the individual teams where the stats are columns and the row index is years.

In order to get there, I can start by taking the data the way it comes, individual years of data with tables (dataframes) for standings/team record, batting, pitching, WAR, fielding. I can groupby league on these tables to get summary data rather than reading in the separate league pages... and in fact I can neglect this entirely for now.

Let's tinker with the first table and start building up a dictionary of dictionaries where the top level key is the year / season and the second level key is the table caption.

In [7]:
print(season_soup[1901].find('table').find('caption').text)

MLB  Detailed Standings


In [8]:
for table in season_soup[1901].find_all('table'):
    print(table.find('caption').text)

MLB  Detailed Standings
Team Standard Batting Table
Team Standard Pitching Table
MLB Wins Above Avg By Position Table
Team Fielding Table


Bingo. Once I stop botching it, Joe's code saves the day. Huzzah for Python string library routines.

In [9]:
season_dict={}
for year, season in season_soup.items():
    season_dict[year]={}
    for table in season.find_all('table'):
        if table.find('caption').text in ['Postseason',
                                          'MLB  Detailed Standings',
                                          'MLB Wins Above Avg By Position Table']:
            continue
        list_holder = []
        rows = table.find_all('tr')
        columns = [thing.text for thing in rows[0].find_all('th')]
        rows.pop(0)
        # The last row is blank.
        rows.pop(-1)
        for team in rows:
            row = [team.find('th').text]
            for thing in team.find_all('td'):
                row.append(thing.text)
            list_holder.append(row)
        df = pd.DataFrame(list_holder,columns=columns)
        # I want to be able to call by team abbreviation
        # rather than boolean indexing below
        df.set_index('Tm',inplace=True)
        season_dict[year][table.find('caption').text] = df
print(season_dict.keys())

dict_keys([1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])


In [10]:
print(season_dict[1968].keys())

dict_keys(['Team Standard Batting Table', 'Team Standard Pitching Table', 'Team Fielding Table'])


In [11]:
print(season_dict[1969].keys())

dict_keys(['Team Standard Batting Table', 'Team Standard Pitching Table', 'Team Fielding Table'])


In [12]:
print(season_dict[2017]['Team Standard Batting Table'].columns)

Index(['#Bat', 'BatAge', 'R/G', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'TB',
       'GDP', 'HBP', 'SH', 'SF', 'IBB', 'LOB'],
      dtype='object')


In [13]:
test_df = season_dict[2017]['Team Standard Batting Table']
test_df.head()

,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,3B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
Tm,,,,,,,,,,,,,,,,,,,,,
ARI,45,28.3,5.01,162,6224,5525,812,1405,314,39,...,.445,.774,94,2457,106,54,39,27,44,1118
ATL,49,28.7,4.52,162,6216,5584,732,1467,289,26,...,.412,.738,92,2303,137,66,59,32,57,1127
BAL,50,28.6,4.59,162,6140,5650,743,1469,269,12,...,.435,.747,100,2458,138,50,10,37,12,1041
BOS,49,27.3,4.85,162,6338,5669,785,1461,302,19,...,.407,.736,92,2305,141,53,9,36,48,1134
CHC,47,27.1,5.07,162,6283,5496,822,1402,274,29,...,.437,.775,99,2403,134,82,48,32,54,1147


In [14]:
print(season_dict[2017]['Team Standard Pitching Table'].columns)

Index(['#P', 'PAge', 'RA/G', 'W', 'L', 'W-L%', 'ERA', 'G', 'GS', 'GF', 'CG',
       'tSho', 'cSho', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO',
       'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9',
       'SO9', 'SO/W', 'LOB'],
      dtype='object')


In [15]:
print(season_dict[2017]['Team Fielding Table'].columns)

Index(['#Fld', 'RA/G', 'DefEff', 'G', 'GS', 'CG', 'Inn', 'Ch', 'PO', 'A', 'E',
       'DP', 'Fld%', 'Rtot', 'Rtot/yr', 'Rdrs', 'Rdrs/yr', 'Rgood'],
      dtype='object')


Now that we have dataframes the way THEY want to format them, I want dataframes the way *I* want to format them.

If I were being really sophisticated, I would try to dynamically allocate new columns, but that makes the loop and branch structure so absurdly complicated. Instead I will initialize the three dataframes with the 1962 column names in a separate batch of code then run through the whole dictionary and hope for the best.

In [16]:
bat_columns = list(season_dict[1962]['Team Standard Batting Table'].columns)
pitch_columns = list(season_dict[1962]['Team Standard Pitching Table'].columns)
field_columns = list(season_dict[1962]['Team Fielding Table'].columns)

What I can't avoid is adding new team dataframes as teams come and go.

Let me think through a workflow:

New season dictionary.

-- Iterate through batting, pitching, fielding dataframes:

--- Iterate through each team:

---- Add a new team dataframe if necessary.
    Assign Season = [year]
    The row of team data is the data.
    Set Season = [year] to be the index.
    I have the column names.

---- If team already exists, append a row of stats for each team.

In [17]:
bat_dict = {}
pitch_dict = {}
field_dict = {}
for key, df in season_dict[1962].items():
    year = 1962
    teams = list(df.index)
    if key == 'Team Standard Batting Table':
        for team in teams:
            data = df.loc[team]
            if team not in bat_dict.keys():
                Season = [year]
                bat_dict[team] = pd.DataFrame(data=None,index=Season,columns=bat_columns)
            bat_dict[team].loc[year] = data
            
print(bat_dict['LgAvg'])

     #Bat BatAge   R/G    G    PA    AB    R     H   2B  3B  ...   SLG   OPS  \
1962   35   27.9  4.46  162  6228  5534  723  1426  216  43  ...  .393  .719   

     OPS+    TB  GDP HBP  SH  SF IBB   LOB  
1962   93  2177  124  35  68  42  41  1154  

[1 rows x 28 columns]


In [19]:
bat_dict = {}
pitch_dict = {}
field_dict = {}

for year, season_data_dict in season_dict.items():
    for key, df in season_data_dict.items():
        teams = list(df.index)
        if key == 'Team Standard Batting Table':
            for team in teams:
                data = df.loc[team]
                if team not in bat_dict.keys():
                    Season = [year]
                    bat_dict[team] = pd.DataFrame(data=None,index=Season,columns=bat_columns)
                bat_dict[team].loc[year] = data
        elif key == 'Team Standard Pitching Table':
            for team in teams:
                data = df.loc[team]
                if team not in pitch_dict.keys():
                    Season = [year]
                    pitch_dict[team] = pd.DataFrame(data=None,index=Season,columns=pitch_columns)
                pitch_dict[team].loc[year] = data
        elif key == 'Team Fielding Table':
            for team in teams:
                data = df.loc[team]
                if team not in field_dict.keys():
                    Season = [year]
                    field_dict[team] = pd.DataFrame(data=None,index=Season,columns=field_columns)
                field_dict[team].loc[year] = data

In [20]:
print(pitch_dict['LgAvg'].head(3))

      #P  PAge  RA/G   W   L  W-L%   ERA    G   GS  GF  ...    BF ERA+   FIP  \
1901   9  26.4  4.99  68  68  .500  3.49  139  139  19  ...  5219  100  3.49   
1902  10  26.6  4.43  68  68  .500  3.17  140  139  17  ...  5160  101  3.17   
1903   9  27.2  4.44  69  69  .500  3.11  139  139  20  ...  5102  100  3.11   

       WHIP   H9  HR9  BB9  SO9  SO/W  LOB  
1901  1.352  9.7  0.2  2.5  3.2  1.28  NaN  
1902  1.307  9.3  0.2  2.5  3.0  1.22  NaN  
1903  1.284  9.1  0.2  2.5  3.6  1.46  NaN  

[3 rows x 35 columns]


In [21]:
print(field_dict['MON'].head(3))

     #Fld  RA/G DefEff    G    GS    CG      Inn    Ch    PO     A    E   DP  \
1969   42  4.88   .684  162  1458  1180  12834.0  6246  4278  1784  184  179   
1970   38  4.98   .697  162  1458  1053  12948.0  6208  4316  1751  141  193   
1971   33  4.50   .698  162  1458  1035  12909.0  6265  4303  1812  150  164   

      Fld% Rtot Rtot/yr  
1969  .971  -53      -5  
1970  .977   22       2  
1971  .976    4       0  


In [22]:
print(season_dict[1901]['Team Standard Batting Table'].tail(3))

      #Bat BatAge   R/G    G    PA    AB    R     H   2B  3B  ...   SLG   OPS  \
Tm                                                            ...               
STL     25   28.8  5.58  142  5565  5039  792  1430  187  94  ...  .381  .718   
WSH     19   28.5  4.95  138  5259  4772  683  1282  191  83  ...  .364  .690   
LgAvg   23   28.1  4.99  139  5317  4819  692  1310  183  77  ...  .360  .686   

      OPS+    TB GDP HBP   SH SF IBB LOB  
Tm                                        
STL    112  1922      90  122             
WSH     92  1738      51   80             
LgAvg   94  1733      52  105             

[3 rows x 28 columns]


In [23]:
with open('batting_data.pkl','wb') as cellar:
    pickle.dump(bat_dict, cellar)
with open('pitching_data.pkl','wb') as cellar:
    pickle.dump(pitch_dict, cellar)
with open('fielding_data.pkl','wb') as cellar:
    pickle.dump(field_dict, cellar)